In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Mart_POS_2015.csv")
df.head(2)

,Member_number,Date,itemDescription
0,1220,2015-01-01,canned beer
1,1220,2015-01-01,margarine


In [3]:
df_list = pd.read_csv("item_list.csv")
df_list.head(2)

,prod_id,prod_nm,alcohol,frozen
0,1,tropical fruit,0,0
1,2,whole milk,0,0


### [[ 문제 1 ]]

In [4]:
df_date = df["Date"].value_counts().reset_index()
df_date.head(2)

,index,Date
0,2015-01-21,96
1,2015-07-21,93


In [5]:
df_date.loc[df_date["Date"] == df_date["Date"].max(), ]

,index,Date
0,2015-01-21,96


In [6]:
df_sub = df.loc[df["Date"] == "2015-01-21", ]
df_sub.head(2)

,Member_number,Date,itemDescription
1087,1081,2015-01-21,tropical fruit
1088,1081,2015-01-21,whole milk


In [7]:
df_sub_cnt = df_sub["itemDescription"].value_counts().reset_index()
df_sub_cnt = df_sub_cnt.sort_values("itemDescription", ascending = False)
df_sub_cnt.head(2)

,index,itemDescription
0,soda,7
1,tropical fruit,5


In [8]:
# 7

### [[ 문제 2 ]]

In [9]:
df_join = df.merge(df_list, left_on = "itemDescription", right_on = "prod_nm", how = "left")
df_join.head(2)

,Member_number,Date,itemDescription,prod_id,prod_nm,alcohol,frozen
0,1220,2015-01-01,canned beer,25,canned beer,1,0
1,1220,2015-01-01,margarine,70,margarine,0,0


In [10]:
df_join_agg = df_join.groupby("Date")["alcohol"].sum().reset_index()
df_join_agg.head(2)

,Date,alcohol
0,2015-01-01,4
1,2015-01-02,3


In [11]:
df_join_agg["Date" ] = pd.to_datetime(df_join_agg["Date"])
df_join_agg["month"] = df_join_agg["Date"].dt.month
df_join_agg["wday" ] = df_join_agg["Date"].dt.weekday
df_join_agg.head()

,Date,alcohol,month,wday
0,2015-01-01,4,1,3
1,2015-01-02,3,1,4
2,2015-01-03,2,1,5
3,2015-01-04,4,1,6
4,2015-01-05,5,1,0


In [12]:
df_join_agg = df_join_agg.loc[df_join_agg["month"].isin([1, 2, 3]), ]

In [13]:
df_join_agg["is_FS"] = df_join_agg["wday"].isin([4, 5]) + 0
df_join_agg.head(2)

,Date,alcohol,month,wday,is_FS
0,2015-01-01,4,1,3,0
1,2015-01-02,3,1,4,1


In [14]:
len(df_join_agg)

90

In [15]:
from scipy.stats import ttest_ind

In [16]:
stat, p = ttest_ind(df_join_agg.loc[df_join_agg["is_FS"] == 0, "alcohol"],
                    df_join_agg.loc[df_join_agg["is_FS"] == 1, "alcohol"], 
                    equal_var = False)
round(p, 3)

0.023

### [[ 문제 3 ]]

In [18]:
df_item = df["itemDescription"].value_counts().reset_index()
df_item = df_item.sort_values("itemDescription", ascending = False)
df_item.head(10)

,index,itemDescription
0,whole milk,1464
1,other vegetables,1039
2,rolls/buns,879
3,sausage,744
4,soda,737
5,yogurt,694
6,tropical fruit,668
7,root vegetables,592
8,citrus fruit,544
9,frankfurter,505


In [19]:
top_10 = df_item.iloc[0:10, 0]
top_10

0          whole milk
1    other vegetables
2          rolls/buns
3             sausage
4                soda
5              yogurt
6      tropical fruit
7     root vegetables
8        citrus fruit
9         frankfurter
Name: index, dtype: object

In [20]:
df["top_10"] = df["itemDescription"].isin(top_10) + 0
df_agg = df.groupby("Date")["top_10"].sum().reset_index()
df_agg.head()

,Date,top_10
0,2015-01-01,16
1,2015-01-02,13
2,2015-01-03,28
3,2015-01-04,21
4,2015-01-05,30


In [21]:
df_agg["Date"] = pd.to_datetime(df_agg["Date"])
df_agg["wday"] = df_agg["Date"].dt.weekday
df_agg.head(2)

,Date,top_10,wday
0,2015-01-01,16,3
1,2015-01-02,13,4


In [22]:
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [23]:
model = ols(formula = "top_10 ~ C(wday)", data = df_agg).fit()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
C(wday),6.0,219.527473,36.587912,0.86853,0.518128
Residual,357.0,15039.076923,42.126266,NaN,NaN


In [24]:
round(0.518128, 2)

0.52